## Dynamic receiving

Process 0 sends a random amount of numbers to a receiver and the receiver then finds out how many numbers were sent. Instead of receiving and simply providing a large buffer to handle all possible sizes of messages, we use `MPI_Probe` to query the message size before actually receiving it. 

In [ ]:
?MPI::MPI_Probe

In [ ]:
?MPI::MPI_Get_count

***
#### C solution

In [ ]:
%%file recv.c
#include <stdio.h>
#include <mpi.h>

int main()
{
    int rank;
    MPI_Init(NULL, NULL);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    int number_amount;
    if (rank == 0) {
        const int MAX_NUMBERS = 100;
        int numbers[MAX_NUMBERS];
        // Pick a random amount of integers to send to process 1
        srand(time(NULL));
        number_amount = (rand() / (float)RAND_MAX) * MAX_NUMBERS;

        // Send the amount of integers to process 1
        MPI_Send(numbers, number_amount, MPI_INT, 1, 0, MPI_COMM_WORLD);
        printf("I am %i and I sent %d numbers to 1\n", rank, number_amount);
    }
    else if (rank == 1) {
        MPI_Status status;
        // Probe for an incoming message from process 0
        MPI_Probe(0, 0, MPI_COMM_WORLD, &status);

        // Get the size of the message
        MPI_Get_count(&status, MPI_INT, &number_amount);
        // Allocate a buffer just big enough
        int* number_buf = (int*)malloc(sizeof(int) * number_amount);

        // Receive the message
        MPI_Recv(number_buf, number_amount, MPI_INT, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        printf("I am %i and I dynamically received %d numbers from 0.\n", rank, number_amount);
        free(number_buf);
    }

    MPI_Finalize();
}

In [ ]:
!mpirun -np 2 a.x

***
#### Fortran solution

In [ ]:
%%file recv.f90
program recv
use mpi

integer ( kind = 4 ) error
integer ( kind = 4 ) rank
integer :: number_amount, MAX_NUMBERS
real :: r
integer, allocatable :: numbers(:), number_buf(:)
integer :: status(MPI_STATUS_SIZE)

call MPI_Init(error);
call MPI_Comm_rank(MPI_COMM_WORLD, rank, error);

if (rank .eq. 0) then
    MAX_NUMBERS = 100
    allocate(numbers(0:MAX_NUMBERS-1))
    ! Pick a random amount of integers to send to process 1
    call random_number(r)
    number_amount = floor(MAX_NUMBERS*r)
    
    ! Send the amount of integers to process 1
    call MPI_Send(numbers, number_amount, MPI_INT, 1, 0, MPI_COMM_WORLD, error)
    print *, 'I am', rank, ' and I sent', number_amount, ' numbers to 1'
    deallocate(numbers)
else if (rank .eq. 1) then
    ! Probe for an incoming message from process 0
    call MPI_Probe(0, 0, MPI_COMM_WORLD, status, error);
    
    ! Get the size of the message
    call MPI_Get_count(status, MPI_INT, number_amount, error);
    ! Allocate a buffer just big enough
    allocate(number_buf(0:number_amount-1))
    
    ! Receive the message
    call MPI_Recv(number_buf, number_amount, MPI_INT, 0, 0, MPI_COMM_WORLD, status, error)
    print *, 'I am', rank, ' and I dynamically received', number_amount, ' numbers from 0'
    deallocate(number_buf)
end if

call MPI_Finalize(error)
end

In [ ]:
!mpif90 recv.f90 && mpirun -np 2 --allow-run-as-root a.out